# NgsAMG

AMG Addon package I developed at TU Wien, available on [Github](https://github.com/LukasKogler/NgsAMG)

Features:
* Relatively standard smoothed aggregation for H1
* SA for Elasticity with some non-standard features
* non-standard Monolithic Stokes-AMG (similar to [ParELAG](https://github.com/LLNL/parelag))
* Some reusable building blocks/smoothers

Everything is MPI-parallel, very little shared-memory parallelization, no GPU :(

#### Installation
Primary target is Linux + MPI-parallel NGSolve, but should work without MPI and on MacOS.

Relatively painless cmake install, pip install is also implemented (probably works too)
 > python3 -m pip install git+https://github.com/LukasKogler/NgsAMG.git

#### Content

Serial and parallel version of each notebook that works with MPI/no MPI build, parallel version has limited drawing.

Serial:
* [H1/Elasticity](BasicAMG.ipynb)
* [Stokes](Stokes.ipynb)
* [Smoothers](Smoothers.ipynb)

Parallel:
* [H1/Elasticity](BasicAMGParallelPETSc.ipynb)
* [Stokes](StokesPar.ipynb)
* [Smoothers](SmoothersPar.ipynb)

Itils used in the other books:
* [serial](usrMtgStuff.py)
* [parallel](usrMtgStuffPar.py)
